In [1]:
import pandas as pd
rewards = pd.read_csv('total_rewards.csv')

In [2]:
import pandas as pd
import json
import datasets
# train_instructions = datasets.load_dataset(
#             "json", data_files='/home/ubuntu/RLHF/LLaVA-RLHF-Data/llava_ppo50k-aokvqa12k-vqa10k-subset-with-reward-hacked.json'
#         )
# train_df = pd.DataFrame(train_instructions['train'])

/home/ubuntu/mambaforge-pypy3/envs/llavav2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
import json
import datasets
train_instructions = datasets.load_dataset(
            "json", data_files='/home/ubuntu/RLHF/LLaVA-RLHF-Data/llava_ppo50k-aokvqa12k-vqa10k.json'
        )
train_df = pd.DataFrame(train_instructions['train'])

Found cached dataset json (/home/ubuntu/.cache/huggingface/datasets/json/default-226d9145801c0b24/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
100%|██████████| 1/1 [00:00<00:00, 388.04it/s]


In [4]:
train_df['id_last6'] = train_df['id'].astype(str).str[-6:]
rewards['id_last6'] = rewards.ID.astype(str).str[-6:]

merged_df = pd.merge(train_df, rewards, on='id_last6', how='inner')


In [5]:
import math

def logit_to_probability(logit):
    """
    Converts a logit to a probability.
    :param logit: logit value
    :return: probability
    """
    return 1 / (1 + math.exp(-logit))
merged_df['normalized_reward'] = merged_df['Reward'].apply(lambda x: logit_to_probability(x))


In [6]:
x = merged_df['normalized_reward']
x_min = x.min()
x_max = x.max()
x_std = x.std()
normalized_values = ((x - x_min) / (x_max - x_min))
normalized_values = (normalized_values - 0.5) / (0.5 * x_std)
merged_df['normalized_rewards'] = normalized_values

In [7]:
def add_reward_to_first_turn_conversation(row):
    first_turn = row['conversations'][0]
    reward_value = f'{row["normalized_reward"]:.2f}'
    first_turn['value'] = first_turn['value'] + f' Answer with reward Value: {reward_value}'
    return row['conversations'][:2] # Only the first two

merged_df['conversations'] = merged_df.apply(add_reward_to_first_turn_conversation, axis=1)

In [8]:
merged_df.to_json('/home/ubuntu/RLHF/LLaVA-RLHF-Data/llava_ppo50k-aokvqa12k-vqa10k-subset-with-reward-hacked.json', orient='records')

In [22]:
# check if image token is present
lengths = [len(x) for x in train_df['conversations']]
max(lengths), min(lengths), sum(lengths)/len(lengths)

(94, 2, 4.2005662262684575)

In [19]:
len(train_df.id.unique()) / len(train_df)

0.7467941600977018